This is based on this code: https://github.com/codekansas/keras-language-modeling/blob/master/keras_models.py

In [1]:
#%load_ext autoreload
#%autoreload 2

In [1]:
import numpy as np
import Settings
import logging
import datetime

from collections import defaultdict
from load_data import load_process_essays

## Load and Pre-Process Essays

In [3]:
import pickle
from window_based_tagger_config import get_config
from IdGenerator import IdGenerator as idGen
from CrossValidation import cross_validation
from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings
from Settings import Settings

CV_FOLDS = 5
DEV_SPLIT = 0.1

settings = Settings()
root_folder = settings.data_directory + "SkinCancer/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
training_pickled = settings.data_directory + "SkinCancer/Thesis_Dataset/training.pl"
models_folder = root_folder + "Models/Bi-LSTM/"

config = get_config(training_folder)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [4]:
training_pickled

'/Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/training.pl'

In [5]:
with open(training_pickled, "rb+") as f:
    tagged_essays = pickle.load(f)
len(tagged_essays)

870

In [6]:
import datetime, logging
print("Started at: " + str(datetime.datetime.now()))
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger()

Started at: 2017-05-21 15:57:46.565383


In [7]:
from numpy.random import shuffle
shuffle(tagged_essays)

In [8]:
def get_unique_words(tessays):
    wds = set()
    for essay in tessays:
        for sentence in essay.sentences:
            for word, tags in sentence:
                wds.add(word)
    return wds

In [9]:
%%time

folds = cross_validation(tagged_essays, CV_FOLDS)
diff_counts = []
vd_lens = []
for i, (essays_TD, essays_VD) in enumerate(folds):
    td_wds = get_unique_words(essays_TD)
    vd_wds = get_unique_words(essays_VD)
    diff = vd_wds - td_wds
    diff_counts.append(len(diff))
    vd_lens.append(len(vd_wds))
    print("|TD|={td_len} |VD|={vd_len} |diff|={len_diff}".format(td_len=len(td_wds), vd_len=len(vd_wds), len_diff=len(diff)))
    
print("")
print(float(sum(diff_counts)) / sum(vd_lens))
print("")

|TD|=1552 |VD|=1170 |diff|=30
|TD|=1567 |VD|=1179 |diff|=15
|TD|=1563 |VD|=1155 |diff|=19
|TD|=1570 |VD|=1122 |diff|=12
|TD|=1549 |VD|=1191 |diff|=33

0.01873818119305484

CPU times: user 154 ms, sys: 2.93 ms, total: 157 ms
Wall time: 155 ms


## Load The Test Dataset

In [10]:
test_folder = root_folder + "Test" + "/"
test_config = get_config(test_folder)
test_essays = load_process_essays(**test_config)

218 files found
218 essays processed


In [11]:
test_folder

'/Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Test/'

In [12]:
td_wds = get_unique_words(tagged_essays)
vd_wds = get_unique_words(test_essays)
diff = vd_wds - td_wds
print("|TD|={td_len} |VD|={vd_len} |diff|={len_diff}".format(td_len=len(td_wds), vd_len=len(vd_wds), len_diff=len(diff)))
print(float(len(diff))/len(vd_wds))

|TD|=1582 |VD|=899 |diff|=41
0.04560622914349277
